In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 32
compression = 0.5
dropout_rate = 0.25

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 32
dropout_rate = 0.5
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [21]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 32, 32, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 32, 32)   128         conv2d_53[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 32, 32, 32)   0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_54 

In [0]:
path="weights1.hdf5"
checkpoint = ModelCheckpoint(path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])


In [24]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
          callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 318s 6ms/step - loss: 1.7752 - acc: 0.3302 - val_loss: 4.1575 - val_acc: 0.2211

Epoch 00001: val_acc improved from -inf to 0.22110, saving model to weights1.hdf5
Epoch 2/50
14720/50000 [=======>......................] - ETA: 3:17 - loss: 1.4805 - acc: 0.4486

50000/50000 [==============================] - 299s 6ms/step - loss: 1.4043 - acc: 0.4785 - val_loss: 2.3836 - val_acc: 0.3659

Epoch 00002: val_acc improved from 0.22110 to 0.36590, saving model to weights1.hdf5
Epoch 3/50
31488/50000 [=================>............] - ETA: 1:43 - loss: 1.2602 - acc: 0.5391

50000/50000 [==============================] - 300s 6ms/step - loss: 1.2288 - acc: 0.5525 - val_loss: 3.4577 - val_acc: 0.3757

Epoch 00003: val_acc improved from 0.36590 to 0.37570, saving model to weights1.hdf5
Epoch 4/50
36096/50000 [====================>.........] - ETA: 1:18 - loss: 1.1121 - acc: 0.5977

50000/50000 [==============================] - 300s 6ms/step - loss: 1.0990 - acc: 0.6019 - val_loss: 1.7347 - val_acc: 0.5263

Epoch 00004: val_acc improved from 0.37570 to 0.52630, saving model to weights1.hdf5
Epoch 5/50
37376/50000 [=====================>........] - ETA: 1:11 - loss: 1.0225 - acc: 0.6316

50000/50000 [==============================] - 301s 6ms/step - loss: 1.0142 - acc: 0.6345 - val_loss: 2.4284 - val_acc: 0.4756

Epoch 00005: val_acc did not improve from 0.52630
Epoch 6/50
42624/50000 [========================>.....] - ETA: 41s - loss: 0.9633 - acc: 0.6556

...

50000/50000 [==============================] - 300s 6ms/step - loss: 0.4340 - acc: 0.8493 - val_loss: 1.6531 - val_acc: 0.6972

Epoch 00046: val_acc did not improve from 0.75850
Epoch 47/50
44672/50000 [=========================>....] - ETA: 29s - loss: 0.4310 - acc: 0.8487

50000/50000 [==============================] - 299s 6ms/step - loss: 0.4310 - acc: 0.8487 - val_loss: 2.0226 - val_acc: 0.6657

Epoch 00047: val_acc did not improve from 0.75850
Epoch 48/50
44672/50000 [=========================>....] - ETA: 29s - loss: 0.4247 - acc: 0.8514

50000/50000 [==============================] - 299s 6ms/step - loss: 0.4247 - acc: 0.8519 - val_loss: 1.1539 - val_acc: 0.7459

Epoch 00048: val_acc did not improve from 0.75850
Epoch 49/50
44672/50000 [=========================>....] - ETA: 29s - loss: 0.4251 - acc: 0.8512

50000/50000 [==============================] - 298s 6ms/step - loss: 0.4241 - acc: 0.8521 - val_loss: 2.6300 - val_acc: 0.5857

Epoch 00049: val_acc did not improve from 0.75850
Epoch 50/50
44672/50000 [=========================>....] - ETA: 29s - loss: 0.4188 - acc: 0.8531

50000/50000 [==============================] - 299s 6ms/step - loss: 0.4185 - acc: 0.8529 - val_loss: 1.4537 - val_acc: 0.7167

Epoch 00050: val_acc did not improve from 0.75850


In [0]:
model.load_weights("weights1.hdf5")

In [26]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 27s 3ms/step
Test loss: 0.9592307312965394
Test accuracy: 0.7585


In [27]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')